In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time
import itertools
import click
import numpy as np

#https://stackoverflow.com/questions/8049520/web-scraping-javascript-page-with-python
from requests_html import HTMLSession, AsyncHTMLSession

# https://www.youtube.com/watch?v=U6gbGk5WPws&list=PLzMcBGfZo4-n40rB1XaJ0ak1bemvlqumQ&index=3&ab_channel=TechWithTim
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

In [2]:
years = [str(i) for i in range(1996, 2019)]
genders = ['Men', 'Women']

for year in years:
    for gender in genders:
        
        # get started on search page
        base_url = 'https://chicago-history.r.mikatiming.com/2019/?pid=start&event=ALL_HISTORY&lang=EN_CAP'
        path = 'C:\Program Files (x86)\chromedriver.exe'
        driver = webdriver.Chrome(path)
        driver.get(base_url)
        
        # let load
        time.sleep(2)
        
        # search parameters
        select_year = Select(driver.find_elements(by=By.NAME, value='event_main_group')[-1]).select_by_visible_text(year)
        select_event = Select(driver.find_elements(by=By.NAME, value='event')[-1]).select_by_visible_text('Marathon')
        time.sleep(3)
        select_gender = Select(driver.find_elements(by=By.NAME, value='search[sex]')[-1]).select_by_visible_text(gender)
        select_age = Select(driver.find_elements(by=By.NAME, value='search[age_class]')[-1]).select_by_visible_text('All')
        select_results = Select(driver.find_elements(by=By.NAME, value='num_results')[-1]).select_by_visible_text('1000')
        time.sleep(3)
        
        # click search
        search_button = driver.find_elements(by=By.NAME, value='submit')[-1]
        search_button.click()

        # get total results for category
        soup = BeautifulSoup(driver.page_source)
        total_results = int(soup.find('li', attrs={'class': 'list-group-item'}).text.split()[0])
        
        # lists to hold results
        time_list = []
        age_list = []
        
        # loop through all pages
        for _ in range(total_results // 1000):
            
            # short pause between requests
            time.sleep(.25)
            
            soup = BeautifulSoup(driver.page_source)
            
            # get ages
            divisions = soup.find_all('div', attrs={'class': 'list-field type-age_class'})
            for div in divisions[1:]:
                age_list.append(div.text.split('Division')[1])
            
            # get times
            times = soup.find_all('div', attrs={'class': 'list-field type-time'})
            for t in times[1:]:
                time_list.append(t.text.split('Finish')[1])
            
            
            # next page
            # https://www.browserstack.com/guide/find-element-by-text-using-selenium
            driver.find_element(by=By.XPATH, value="//*[text()='>']").click()

            
        # make df
        df = pd.DataFrame()
        df['time'] = time_list
        df['age'] = age_list
        df['gender'] = gender
        df['year'] = year
        
        #save to csv
        df.to_csv(f'../data/Chicago_Data/Original/Chicago_Results_{year}_{gender}.csv', index=False)
        
        # end selenium instance
        driver.quit()

C:\Users\David\AppData\Local\Temp/ipykernel_22048/1657261884.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)
